In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import cmath
from numba import njit, f8,i8,c16
import numba
import matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
import copy

import sys
sys.path.append('../../lb')
import RungeKutta_acceralated

#パラメーター
nu = 0.001

f = (1 + 1j) * 5 * 0.001

Dimention = 14 #uの次元数
q = 2
k_0 = 2 ** (-4)
k_n = np.zeros((Dimention+4,1))
k_n[2:-2,0] = [k_0 * q ** (i+1) for i in range(Dimention)]
#print(k_n)

c_n_1 = copy.deepcopy(k_n)[2:-2, 0:]
c_n_1[-2:, 0:] = np.zeros((2,1))
#print(c_n_1)

beta = 0.5
c_n_2 = copy.deepcopy(k_n)[1:-3, 0:] * -beta
c_n_2[-1, 0] = np.zeros((1, 1))
#print(c_n_2)

c_n_3 = copy.deepcopy(k_n)[:-4, 0:] * (beta - 1)
#print(c_n_3)

#parameter = [nu,f,k_n,c_n_1,c_n_2,c_n_3]
parameter = {
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}
#print(parameter)
#print(k_n)

runge_para = {
    'start' :  np.array([
        [0.4350E+00 + 0.5008E+00j],
        [0.1259E+00 + 0.2437E+00j],
        [-0.8312E-01 + -0.4802E-01j],
        [0.5164E-01 + -0.1599E+00j],
        [-0.1899E+00 + -0.3602E-01j],
        [0.4093E-03 + 0.8506E-01j],
        [0.9539E-01 + 0.3215E-01j],
        [-0.5834E-01 + 0.4433E-01j],
        [-0.8790E-02 + 0.2502E-01j],
        [0.3385E-02 + 0.1148E-02j],
        [-0.7072E-04 + 0.5598E-04j],
        [-0.5238E-07 + 0.1467E-06j],
        [0.1E-07 + 0.1E-06j],
        [0.1E-07 + 0.1E-06j]
    ]),
    'step' : 0.01,
    'period' : (0,5000),
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}

In [ ]:
%%time
#縦軸に各変数、横軸時間　×次元数
runge_para["period"]=(0, 10000)

#刻み幅
runge_para["step"] = 0.01

skip=1000

latter = 10

runge_para['start'] = np.load("../laminar_initials/beta0.417_nu0.00017322774042408294_step0.01_10000.0period_laminar.npy")

runge_para["nu"] =0.00017322774042408294
beta= 0.417


#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr_latter(**runge_para,latter=latter)

runge_para["start"]=model.arr_latter[:-1,-1:]

In [ ]:
#2点の差を見る(第N次元)
start_error = 1E-5
#時間
runge_para["period"]=(0,10000)

#初期値の誤差をとる次元
N = 4

runge_para['start'] = np.load("../laminar_initials/beta0.419_nu0.000172168_step0.01_5000.0period_laminar.npy")

runge_para["nu"] = 0.000172168
beta= 0.419

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
#1個目
model.get_arr(**runge_para)
start1 = model.arr

#2個目
runge_para["start"][N-1] += start_error
model.get_arr(**runge_para)
start2 = model.arr

error = abs(start2[N-1,:]) - abs(start1[N-1,:])

plt.figure(figsize=(20,8))
plt.ylabel(f"u{N}_error")
plt.yscale=("log")
plt.xlabel("time")
plt.plot(np.real(model.arr[-1,:]),error,lw=1)

plt.savefig(fname=f"../initial sensitivity/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time{runge_para['period']}_{Dimention}dim_error{start_error}.png")

In [ ]:
#軌道(運動アニメーション)
# runge_para["period"]=(0,10000)

# #start_error = 0.0001

#plotする次元
N = (4,5)

# #nu
# runge_para["nu"] = 0.0001760992781527306
# #betaの値に従いc_n_2,c_n_3を更新
# beta= 0.41525252525252526

# #c_n_2
# runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
# runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

# #c_n_3
# runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

# #インスタンス化
# model = RungeKutta_acceralated.RungeKutta()

# #1個目
# model.get_arr(**runge_para)
# start1 = model.arr

# #2個目
# runge_para["start"] += start_error
# model.get_arr(**runge_para)
# start2 = model.arr

#プロットの設定
fig, ax = plt.subplots(dpi=100)
ims = []
ax.set_xlabel(f"|u{N[0]}|")
ax.set_ylabel(f"|u{N[1]}|")
matplotlib.rcParams['animation.embed_limit'] = 10**6

length = 1000

#アニメーション
for i in tqdm(range(len(model.arr[-1,:])//length-1)):
    text = ax.text(0.5,1.05,(f"time={abs(model.arr[-1,length*i])}"), size = 15, color = "green", ha = "center", transform=ax.transAxes)
    
    im1 = ax.plot(abs(start1[N[0]-1,i*length:(i+1)*length]),abs(start1[N[1]-1,i*length:(i+1)*length]),color="b",linewidth=1)
    
    im2 = ax.plot(abs(start2[N[0]-1,i*length:(i+1)*length]),abs(start2[N[1]-1,i*length:(i+1)*length]),color="r",linewidth=1)
    
    ims.append(im1+im2+[text])

anim = animation.ArtistAnimation(fig, ims)
anim.save(f"../initial sensitivity/nu{runge_para['nu']}_beta{beta}_{runge_para['step']}_time{runge_para['period']}_{Dimention}dim_error{start_error}.mp4")
HTML(anim.to_jshtml())